# instalacion

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Librerias

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

# Dataset

In [ ]:
sts_dataset_path = "datasets/stsbenchmark.tsv.gz"
util.http_get("https://sbert.net/datasets/stsbenchmark.tsv.gz", sts_dataset_path)

  0%|          | 0.00/392k [00:00<?, ?B/s]

In [ ]:
# !gunzip /content/datasets/stsbenchmark.tsv.gz

In [ ]:
# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")

train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, "rt", encoding="utf8") as fIn:
    reader = csv.DictReader(fIn, delimiter="\t", quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row["score"]) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row["sentence1"], row["sentence2"]], label=score)

        if row["split"] == "dev":
            dev_samples.append(inp_example)
        elif row["split"] == "test":
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [ ]:
model_name = "nli-distilroberta-base-v2"
model_name = "paraphrase-multilingual-mpnet-base-v2"
train_batch_size = 16
num_epochs = 4
train_batch_size = 16

In [ ]:
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
train_loss

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
            epochs=40,
            warmup_steps=200,
            evaluation_steps= 50,
            max_grad_norm=1.0,
            use_amp=True,
            scheduler='warmuplinear',
            steps_per_epoch=None
            )

Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
model.save("paraphrase-multilingual-mpnet-base-v2-example")

In [ ]:
!zip /content/paraphrase-multilingual-mpnet-base-v2-example.zip -r /content/paraphrase-multilingual-mpnet-base-v2-example

updating: content/paraphrase-multilingual-mpnet-base-v2-example/ (stored 0%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/1_Pooling/ (stored 0%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/1_Pooling/config.json (deflated 57%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/tokenizer.json (deflated 76%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/sentencepiece.bpe.model (deflated 49%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/README.md (deflated 58%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/similarity_evaluation_sts-test_results.csv (deflated 46%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/config.json (deflated 49%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/model.safetensors (deflated 9%)
  adding: content/paraphrase-multilingual-mpnet-base-v2-example/special_tokens_map.json (deflated 85%)
  adding: content/paraphrase-multilin

# Testing

In [ ]:
model = SentenceTransformer('/content/paraphrase-multilingual-mpnet-base-v2-example')
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name="sts-test")

In [ ]:
test_evaluator(model, output_path='/content/paraphrase-multilingual-mpnet-base-v2-example')

0.8660914088309746

In [ ]:
[x.texts for x in test_samples]

[['A girl is styling her hair.', 'A girl is brushing her hair.'],
 ['A group of men play soccer on the beach.',
  'A group of boys are playing soccer on the beach.'],
 ["One woman is measuring another woman's ankle.",
  "A woman measures another woman's ankle."],
 ['A man is cutting up a cucumber.', 'A man is slicing a cucumber.'],
 ['A man is playing a harp.', 'A man is playing a keyboard.'],
 ['A woman is cutting onions.', 'A woman is cutting tofu.'],
 ['A man is riding an electric bicycle.', 'A man is riding a bicycle.'],
 ['A man is playing the drums.', 'A man is playing the guitar.'],
 ['A man is playing guitar.', 'A lady is playing the guitar.'],
 ['A man is playing a guitar.', 'A man is playing a trumpet.'],
 ['A man is playing a guitar.', 'A man is playing a trumpet.'],
 ['A man is cutting an onion.', 'A man cuts an onion.'],
 ['A man is cycling.', 'A man is talking.'],
 ['A man is slicing open a fish.', 'A man is cutting up a fish.'],
 ['A man is slicing a tomato.', 'A man is 

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sklearn.metrics.pairwise import cosine_similarity
# from app.utils.nlp.finetunned_embeddings import FinetunnedEmbModelSentenceTransformer

In [ ]:
model = SentenceTransformer('/content/paraphrase-multilingual-mpnet-base-v2-example')
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
texto_base= "Cómo hago la reposición de plástico o reimpresión de NIP de un cliente en estado de postración"
q

embedding1 = model.encode(texto_base)
embedding2 = model.encode(uery = "reimpresión NIP postración"query)

In [ ]:
similitud = cosine_similarity([embedding1], [embedding2])[0][0]

In [ ]:
similitud

0.75409245

In [ ]:
texto_base= "Cuáles son los datos personales de un cliente"
query = "Me podrías explicar cómo reponer el plástico de un cliente en postración?"

embedding1 = model.encode(texto_base)
embedding2 = model.encode(query)

In [ ]:
similitud = cosine_similarity([embedding1], [embedding2])[0][0]
similitud

0.308846